In [1]:
import bz2
import json
import requests as rq
import time

In [2]:
import lzma

In [77]:
# dump = "/10TBdrive/datashare/Reddit/comments/RC_2017-07.bz2"
dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-10.bz2"

In [78]:
openedDump = bz2.BZ2File(dump, "r")
# openedDump = lzma.open(dump)

In [79]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print(repr(line))
    count+=1

b'{"archived":false,"author":"SealTeamSith","author_flair_css_class":"red","author_flair_text":"Transgender","brand_safe":false,"contest_mode":false,"created_utc":1506816000,"distinguished":null,"domain":"self.RightwingLGBT","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"73ig7j","is_crosspostable":false,"is_reddit_media_domain":false,"is_self":true,"is_video":false,"link_flair_css_class":"rainbow","link_flair_text":"GO VOTE!","locked":false,"media":null,"media_embed":{},"num_comments":20,"num_crossposts":0,"over_18":false,"parent_whitelist_status":null,"permalink":"/r/RightwingLGBT/comments/73ig7j/im_going_to_be_applying_some_css_changes_this/","pinned":false,"retrieved_on":1510382444,"score":7,"secure_media":null,"secure_media_embed":{},"selftext":"","spoiler":false,"stickied":false,"subreddit":"RightwingLGBT","subreddit_id":"t5_3dc8i","suggested_sort":null,"thumbnail":"self","thumbnail_height":null,"thumbnail_width":null,"title":"I\'m going to be applying some CSS 

In [80]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print("Failed to get page")
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print("Invalid Format")
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        print (getSubredditId(json))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['link_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData


def filterSubmissionsbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit_id'] in subID_dict:
            if lineData['subreddit_id'] not in returnData:
                returnData[lineData['subreddit_id']] = []
            postId = lineData['id']
            
            if postId not in returnData[lineData['subreddit_id']]:
                returnData[lineData['subreddit_id']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData



def getSubmissionPermalinks(filePath , comments_Thresh = 20 , limit=None):
    import json
    import bz2
    import lzma
    
    ext = filePath.split('.')[-1]
    
    if ext == 'bz2':
        openedDump = bz2.BZ2File(filePath, "r")
    elif ext == 'xz':
        openedDump = lzma.open(dump)
    else:
        print ("Invalid file type")
    count = 0
    returnData = {}
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if lineData['num_comments'] > comments_Thresh:
            if (limit!=None and count > limit):
                return returnData
            try:
                returnData[lineData['id']] = lineData['permalink']
            except:
                print("Permalink not found, moving on!")
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData

In [81]:
subreddits = ["bipolar" , "BipolarReddit" , "BipolarSOs" , "Health" , "addiction", "depression","SuicideWatch","Anxiety","Chicago"] 

In [82]:
subID_dict = getSubIds(subreddits)

t5_2qnv4
t5_2qzru
t5_2zww6
t5_2qh9z
t5_2qwvc
t5_2qqqf
t5_2qpzs
t5_2qmij
t5_2qh2t


In [83]:
subID_dict

{'Anxiety': 't5_2qmij',
 'BipolarReddit': 't5_2qzru',
 'BipolarSOs': 't5_2zww6',
 'Chicago': 't5_2qh2t',
 'Health': 't5_2qh9z',
 'SuicideWatch': 't5_2qpzs',
 'addiction': 't5_2qwvc',
 'bipolar': 't5_2qnv4',
 'depression': 't5_2qqqf'}

In [84]:
# postData = filterDumpbySub(dump,subID_dict)
postData = getSubmissionPermalinks(dump,comments_Thresh= 10)

=============Parsing============
{'bipolar': 't5_2qnv4', 'BipolarReddit': 't5_2qzru', 'BipolarSOs': 't5_2zww6', 'Health': 't5_2qh9z', 'addiction': 't5_2qwvc', 'depression': 't5_2qqqf', 'SuicideWatch': 't5_2qpzs', 'Anxiety': 't5_2qmij', 'Chicago': 't5_2qh2t'}
Done parsing %d posts in %d seconds (0, 0.015890121459960938)
Done parsing %d posts in %d seconds (100000, 10.112196683883667)
Done parsing %d posts in %d seconds (200000, 10.55633282661438)
Done parsing %d posts in %d seconds (300000, 10.34863543510437)
Done parsing %d posts in %d seconds (400000, 10.129017114639282)
Done parsing %d posts in %d seconds (500000, 10.46405553817749)
Done parsing %d posts in %d seconds (600000, 10.65838885307312)
Done parsing %d posts in %d seconds (700000, 10.296407699584961)
Done parsing %d posts in %d seconds (800000, 10.31222414970398)
Done parsing %d posts in %d seconds (900000, 10.640945196151733)
Done parsing %d posts in %d seconds (1000000, 10.38311505317688)
Done parsing %d posts in %d second

In [85]:
len(postData)

1338231

In [87]:
postData

KeyError: 1

In [86]:
import pickle as pkl
pkl.dump(file=open('/10TBdrive/datashare/Reddit/Submissions102017_atleast10.pkl','wb'),obj=postData)

In [89]:
postData[list(postData.keys())[1]]

'/r/edc_raffle/comments/73ig7k/omega_seamaster_aqua_terra_quartz_65_spots_at/'

In [90]:
# import pickle as pkl
# with open("/datasets_1/sagarj/IoPPN_collab/MentalHealth_2017_posts.pkl",'w') as f:
#     pkl.dump(postData,f)

In [91]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [92]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)

In [103]:
from glob import glob
from tqdm import tqdm

In [112]:
geoFiles = glob('/10TBdrive/datashare/COVID-19-Reddit/download_Reddit/US_permalinks_geo_results/*.json')

In [115]:
with open(geoFiles[1],'r') as f:
    posts = f.readlines()

In [116]:
p = json.loads(posts[1])
p

{'author': 'owledge',
 'permalink': '/r/nba/comments/7nbxvz/game_thread_philadelphia_76ers_1619_phoenix_suns/'}

In [126]:
year_list = ['2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019']

In [127]:
root_dir = '/10TBdrive/datashare/geo_located_Reddit_crawl/yearlySubmissions/'
for year in year_list:
    geoPosts = {}
    for f in tqdm(geoFiles):
        if f.find(year)>1:
            with open(f,'r') as f:
                posts = f.readlines()
                for p in posts:
                    dictionary = json.loads(p)
                    link = dictionary['permalink']
                    ID = link.split('/')[4]
                    geoPosts[ID] = link
    pkl.dump( geoPosts , open(root_dir+year+'.pkl','wb'))



  0%|          | 0/171 [00:00<?, ?it/s]

 26%|██▌       | 44/171 [00:00<00:00, 385.53it/s]

100%|██████████| 171/171 [00:00<00:00, 1185.69it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

 29%|██▉       | 50/171 [00:00<00:00, 422.27it/s]

 96%|█████████▌| 164/171 [00:00<00:00, 499.14it/s]

100%|██████████| 171/171 [00:00<00:00, 674.49it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

 23%|██▎       | 40/171 [00:00<00:00, 304.83it/s]

 33%|███▎      | 57/171 [00:00<00:00, 216.33it/s]

 65%|██████▌   | 112/171 [00:00<00:00, 257.38it/s]

 78%|███████▊  | 134/171 [00:00<00:00, 145.80it/s]

100%|██████████| 171/171 [00:00<00:00, 219.98it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

  8%|▊         | 13/171 [00:00<00:01, 84.73it/s]

 12%|█▏        | 20/171 [00:00<00:02, 69.69it/s]

 15%|█▍        | 25/171 [00:00<00:03, 45.13it/s]

 20%|█▉        | 34/171 [00:00<00:03, 45.66it/s]

 33%|███▎      | 56/171 [00:00<00:02, 53.54it/s]

 37%|███▋      | 63/171 [00:01<00:02, 40.58it/s]

 40%|████      | 

 44%|████▍     | 75/171 [04:28<13:28,  8.42s/it]

 47%|████▋     | 81/171 [05:08<11:52,  7.92s/it]

 76%|███████▌  | 130/171 [05:40<03:55,  5.74s/it]

 89%|████████▉ | 152/171 [06:11<01:24,  4.45s/it]

 95%|█████████▍| 162/171 [06:42<00:36,  4.01s/it]

100%|██████████| 171/171 [06:42<00:00,  2.35s/it]

  0%|          | 0/171 [00:00<?, ?it/s]

  1%|          | 2/171 [00:33<46:34, 16.53s/it]

  5%|▌         | 9/171 [01:16<36:14, 13.42s/it]

  8%|▊         | 14/171 [01:55<30:49, 11.78s/it]

 13%|█▎        | 22/171 [02:40<24:40,  9.93s/it]

 39%|███▊      | 66/171 [03:21<12:39,  7.23s/it]

 51%|█████     | 87/171 [04:09<08:02,  5.74s/it]

 67%|██████▋   | 115/171 [04:53<04:11,  4.49s/it]

 85%|████████▍ | 145/171 [05:39<01:33,  3.61s/it]

 85%|████████▌ | 146/171 [06:22<06:22, 15.30s/it]

 97%|█████████▋| 166/171 [06:56<00:56, 11.22s/it]

 98%|█████████▊| 168/171 [07:32<00:39, 13.24s/it]

100%|██████████| 171/171 [08:04<00:00, 12.49s/it]

  0%|          | 0/171 [00:00<?, ?it/s]

 30%|███  

In [128]:
len(geoPosts)

24111329

In [114]:
del geoPosts